In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import numpy as np
import sklearn.cluster as c
import pickle
import glob

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/data/clean_data_wo_outliers.csv')
df['Nace2'] = df['Nace2'].astype(int)
df['gr_4digits'] = df['gr_4digits'].astype(int)

In [44]:
df.Nace2.nunique(), df.gr_4digits.nunique(), df.gr_stat_letter.nunique(), df.gr_stat_code.nunique()

(769, 595, 20, 82)

In [42]:
from sklearn.preprocessing import LabelEncoder

In [45]:
encoded_labels_df = pd.DataFrame()

encoder = LabelEncoder()

class_20 = encoder.fit_transform(df['gr_stat_letter'])
with open('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/Helpers/class_20.pkl', 'wb') as f:
  pickle.dump(encoder, f)
encoded_labels_df['class_20'] = class_20

class_82 = encoder.fit_transform(df['gr_stat_code'])
with open('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/Helpers/class_82.pkl', 'wb') as f:
  pickle.dump(encoder, f)
encoded_labels_df['class_82'] = class_82

class_595 = encoder.fit_transform(df['gr_4digits'])
with open('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/Helpers/class_595.pkl', 'wb') as f:
  pickle.dump(encoder, f)
encoded_labels_df['class_595'] = class_595

class_769 = encoder.fit_transform(df['Nace2'])
with open('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/Helpers/class_769.pkl', 'wb') as f:
  pickle.dump(encoder, f)
encoded_labels_df['class_769'] = class_769

In [47]:
encoded_labels_df.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/Helpers/encoded_labels_df.csv', index=False)

In [4]:
ae_bert = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/AE/bert_ae_embeddings.csv')
ae_labse = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/AE/labse_ae_embeddings.csv')
ae_w2v = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/AE/w2v_ae_embeddings.csv')

pca_bert = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/PCA/bert_pca_embeddings.csv')
pca_labse = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/PCA/labse_pca_embeddings.csv')
pca_w2v = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/PCA/w2v_pca_embeddings.csv')

svd_bert = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/SVD/bert_svd_embeddings.csv')
svd_labse = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/SVD/labse_svd_embeddings.csv')
svd_w2v = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/SVD/w2v_svd_embeddings.csv')

umap_bert = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/UMAP/bert_umap_embeddings.csv')
umap_labse = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/UMAP/labse_umap_embeddings.csv')
umap_w2v = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/UMAP/w2v_umap_embeddings.csv')

datasets = [ae_bert, ae_labse, ae_w2v, pca_bert, pca_labse, pca_w2v,
            svd_bert, svd_labse, svd_w2v, umap_bert, umap_labse, umap_w2v]
dataset_names = ['AutoEncoder BERT', 'AutoEncoder LaBSE', 'AutoEncoder W2V',
                 'PCA BERT', 'PCA LaBSE', 'PCA W2V',
                 'SVD BERT', 'SVD LaBSE', 'SVD W2V',
                 'UMAP BERT', 'UMAP LaBSE', 'UMAP W2V']

In [5]:
for dataset, dataset_name in zip(datasets, dataset_names):
  zeroes = dataset.columns[(dataset == 0).all()]
  if np.any(zeroes):
    print(f"{dataset_name} has zeros columns", end=' | ')
    print(f"Dropping zero columns", end=' | ')
    dataset.drop(zeroes, axis=1, inplace=True)
    print(f"Zero columns dropped")

AutoEncoder BERT has zeros columns | Dropping zero columns | Zero columns dropped
AutoEncoder LaBSE has zeros columns | Dropping zero columns | Zero columns dropped
AutoEncoder W2V has zeros columns | Dropping zero columns | Zero columns dropped


# Preparing model splits

In [16]:
kmeans_20, kmeans_82, kmeans_595, kmeans_769 = [], [], [], []
agg_20, agg_82, agg_595, agg_769 = [], [], [], []
gmm_20, gmm_82, gmm_595, gmm_769 = [], [], [], []
meanshift, optics, birch, hdbscan = [], [], [], []

In [ ]:
CLUSTERS_PATH = '/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/clustering_outputs/models/'
RESULT_PATH = '/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/results/'

In [13]:
all_model_paths = glob.glob(CLUSTERS_PATH+'**/*.pkl')

In [17]:
for path in all_model_paths:
  if 'kmeans' in path:
    if '20' in path:
      kmeans_20.append(path)
    elif '82' in path:
      kmeans_82.append(path)
    elif '595' in path:
      kmeans_595.append(path)
    elif '769' in path:
      kmeans_769.append(path)
  elif 'agglomerative_clustering' in path:
    if '20' in path:
      agg_20.append(path)
    elif '82' in path:
      agg_82.append(path)
    elif '595' in path:
      agg_595.append(path)
    elif '769' in path:
      agg_769.append(path)
  elif 'mean_shift' in path:
    meanshift.append(path)
  elif 'optics' in path:
    optics.append(path)
  elif 'birch' in path:
    birch.append(path)
  elif 'hdbscan' in path:
    hdbscan.append(path)
  elif 'gmm' in path:
    if '20' in path:
      gmm_20.append(path)
    elif '82' in path:
      gmm_82.append(path)
    elif '595' in path:
      gmm_595.append(path)
    elif '769' in path:
      gmm_769.append(path)

# Result Evaluation

In [25]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import mutual_info_score, adjusted_rand_score

## Hard Clustering Evaluation

In [48]:
def calculate_results(model_paths, num_clustes, target_column_name,
                      has_labels_attr):
  all_results = []
  for dataset, dataset_name, model_path in zip(datasets, dataset_names, model_paths):
    with open(model_path, 'rb') as f:
      model = pickle.load(f)

    if not has_labels_attr:
      y_preds = model.predict(dataset)
    else:
      y_preds = model.labels_

    silhouette = silhouette_score(dataset, y_preds)
    db_index = davies_bouldin_score(dataset, y_preds)
    ch_index = calinski_harabasz_score(dataset, y_preds)
    ari = adjusted_rand_score(encoded_labels_df[target_column_name], y_preds)
    mi = mutual_info_score(encoded_labels_df[target_column_name], y_preds)

    single_result = {
        'dataset_name': dataset_name,
        'model': model_path.split('/')[-2],
        'num_clusters': num_clustes,
        'silhouette': silhouette,
        'db_index': db_index,
        'ch_index': ch_index,
        'ari': ari,
        'mi': mi
    }

    all_results.append(single_result)
  return all_results

In [28]:
models_to_run = [kmeans_20, kmeans_82, kmeans_595, kmeans_769,
                 agg_20, agg_82, agg_595, agg_769, gmm_20, gmm_82,
                 gmm_595, gmm_769]

In [50]:
results = []

for models in models_to_run:
  has_labels_attr = True
  num_clusters = int(kmeans_20[0].split('/')[-1].split('_')[-1].split('.')[0])
  target_column_name = 'class_'+str(num_clusters)
  if models[0].split('/')[-2] == 'gmm': has_labels_attr = False

  results.extend(calculate_results(models, num_clusters, target_column_name,
                                   has_labels_attr))


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/usr/local

In [51]:
hard_cluster_res = pd.DataFrame(results)

In [53]:
hard_cluster_res.to_csv(RESULT_PATH+'hard_clustering_res.csv', index=False)

## Soft Clustering evaluation

In [58]:
models_to_run = [meanshift, optics, birch, hdbscan]

In [63]:
def calculate_results(model_paths):
  all_results = []
  for dataset, dataset_name, model_path in zip(datasets, dataset_names, model_paths):
    with open(model_path, 'rb') as f:
      model = pickle.load(f)

    y_preds = model.labels_
    try:
      silhouette = silhouette_score(dataset, y_preds)
      db_index = davies_bouldin_score(dataset, y_preds)
      ch_index = calinski_harabasz_score(dataset, y_preds)

      single_result = {
          'dataset_name': dataset_name,
          'model': model_path.split('/')[-2],
          'silhouette': silhouette,
          'db_index': db_index,
          'ch_index': ch_index,
      }

      all_results.append(single_result)
    except Exception:
      print(f"Error model {model_path.split('/')[-2]} - Dataset name {dataset_name}")
  return all_results

In [64]:
results = []

for models in models_to_run:
  results.extend(calculate_results(models))


Error model mean_shift - Dataset name AutoEncoder BERT
Error model mean_shift - Dataset name AutoEncoder LaBSE
Error model mean_shift - Dataset name PCA BERT
Error model mean_shift - Dataset name PCA LaBSE
Error model mean_shift - Dataset name SVD BERT
Error model mean_shift - Dataset name SVD LaBSE
Error model birch - Dataset name AutoEncoder BERT
Error model birch - Dataset name AutoEncoder LaBSE
Error model birch - Dataset name PCA LaBSE
Error model birch - Dataset name SVD LaBSE


In [65]:
soft_cluster_res = pd.DataFrame(results)
soft_cluster_res.to_csv(RESULT_PATH+'soft_clustering_res.csv', index=False)

In [73]:
hard_cluster_res.sort_values(by='db_index', ascending=True)

,dataset_name,model,num_clusters,silhouette,db_index,ch_index,ari,mi
23,UMAP W2V,kmeans,20,0.337323,0.694621,10883.969989,0.059727,0.873567
58,UMAP LaBSE,agglomerative_clustering,20,0.360552,0.696184,9087.153992,0.153991,0.828450
71,UMAP W2V,agglomerative_clustering,20,0.295696,0.725940,9705.420346,0.060730,0.865191
10,UMAP LaBSE,kmeans,20,0.366893,0.799440,8890.566837,0.165173,0.864913
119,UMAP W2V,gmm,20,0.264743,0.803291,8979.312411,0.058051,0.881246
...,...,...,...,...,...,...,...,...
66,SVD BERT,agglomerative_clustering,20,0.019176,3.433638,200.447305,0.041148,0.819536
51,PCA BERT,agglomerative_clustering,20,0.026240,3.439424,513.444312,0.073186,0.528831
99,PCA BERT,gmm,20,0.025371,3.481617,513.414274,0.086296,0.633756
55,SVD LaBSE,agglomerative_clustering,20,0.041124,3.484320,423.966827,0.094607,0.728777
